In [2]:
from datasets import load_dataset
from tqdm import tqdm
from knowledge_propagation.utils import vars, io
from scipy.stats import describe

/home/zliu/miniconda3/envs/cpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
rc = load_dataset("/home/zliu/shared_resources/datasets/trivia_qa", 'rc.wikipedia') # 'rc.web'

Generating test split: 100%|██████████| 7701/7701 [00:02<00:00, 2821.63 examples/s]


In [6]:
non_question_ends = []
multiple_question_mark = []
for split in ["train", "validation", "test"][:]:
    clean_sft_data = []
    
    for datum in tqdm(rc[split],):
        q = datum["question"]
        a = datum["answer"]["value"]
        if q[0] == q[-1] == "\"" or q[0] == q[-1] == "\'":
            q = q[1:-1]
        
        if not q.endswith("?"):
            non_question_ends.append(q)
            continue
        
            
        t = f"{q}{a}" if a[0] == " " else f"{q} {a}"
        if t.count("?") != 1:
            multiple_question_mark.append(q)
            continue
        clean_sft_data.append(
            {
                "question": q,
                "answer": a,
                "text": t,
            }
        )
    print("Split =", split)
    print("# multiple_question_mark ", len(multiple_question_mark))
    print("# not ending in `?`", len(non_question_ends))
    print("# clean instances", len(clean_sft_data))
    io.dump_jsonlines(clean_sft_data, f"{vars.DATA_DIR}/trivia_qa_wiki_sft/{split[:5]}.jsonl") 
    

100%|██████████| 61888/61888 [00:10<00:00, 6183.07it/s]


Split = train
# multiple_question_mark  310
# not ending in `?` 3039
# clean instances 58539


100%|██████████| 7993/7993 [00:01<00:00, 6459.62it/s]


Split = validation
# multiple_question_mark  353
# not ending in `?` 3458
# clean instances 7531


100%|██████████| 7701/7701 [00:01<00:00, 7628.68it/s]


Split = test
# multiple_question_mark  399
# not ending in `?` 3868
# clean instances 7245


In [58]:
print(len(multiple_question_mark))

297


In [ ]:
# {"question": , "answer": , "text": }
rc

DatasetDict({
    train: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 138384
    })
    validation: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 17944
    })
    test: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 17210
    })
})

In [8]:
describe([len(vars.GPT_4_TOKENIZER(x["text"])) for x in io.load_jsonlines(f"{vars.DATA_DIR}/trivia_qa_wiki_sft/train.jsonl")])

DescribeResult(nobs=58539, minmax=(np.int64(7), np.int64(137)), mean=np.float64(21.507285741129845), variance=np.float64(87.2176644680619), skewness=np.float64(1.7815887394080325), kurtosis=np.float64(5.913777618588101))

0

True